# REPORT

``` html

Glove.6B.news

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 198s - loss: 1.3827 - acc: 0.5370 - val_loss: 0.2562 - val_acc: 0.9187
Epoch 2/2
15998/15998 [==============================] - 203s - loss: 0.1973 - acc: 0.9359 - val_loss: 0.1361 - val_acc: 0.9485

12 epoches:

Epoch 10/12
15998/15998 [==============================] - 207s - loss: 0.0624 - acc: 0.9701 - val_loss: 0.1400 - val_acc: 0.9442
Epoch 11/12
15998/15998 [==============================] - 208s - loss: 0.0612 - acc: 0.9697 - val_loss: 0.1580 - val_acc: 0.9437
Epoch 12/12
15998/15998 [==============================] - 206s - loss: 0.0581 - acc: 0.9714 - val_loss: 0.1462 - val_acc: 0.9490

not-borrow pretrained

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 266s - loss: 2.3430 - acc: 0.2018 - val_loss: 1.6043 - val_acc: 0.4079
Epoch 2/2
15998/15998 [==============================] - 261s - loss: 0.9845 - acc: 0.6658 - val_loss: 0.5620 - val_acc: 0.8250

adam + pretrained

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 205s - loss: 1.7845 - acc: 0.4146 - val_loss: 0.3782 - val_acc: 0.8627
Epoch 2/2
15998/15998 [==============================] - 210s - loss: 0.2179 - acc: 0.9318 - val_loss: 0.1643 - val_acc: 0.9465

adam + batchsized = 64

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 217s - loss: 1.1108 - acc: 0.6467 - val_loss: 0.1907 - val_acc: 0.9395
Epoch 2/2
15998/15998 [==============================] - 207s - loss: 0.1458 - acc: 0.9489 - val_loss: 0.1498 - val_acc: 0.9450

adam + batchsized = 2x (256)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 209s - loss: 2.2470 - acc: 0.2920 - val_loss: 0.9267 - val_acc: 0.6992
Epoch 2/2
15998/15998 [==============================] - 210s - loss: 0.4158 - acc: 0.8754 - val_loss: 0.2180 - val_acc: 0.9252

rmsprop + 256

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 208s - loss: 2.0523 - acc: 0.3336 - val_loss: 1.0941 - val_acc: 0.6427
Epoch 2/2
15998/15998 [==============================] - 208s - loss: 0.4382 - acc: 0.8507 - val_loss: 0.3604 - val_acc: 0.8637


Glove.27B.tweet

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 297s - loss: 1.2132 - acc: 0.6106 - val_loss: 0.3985 - val_acc: 0.8702
Epoch 2/2
15998/15998 [==============================] - 296s - loss: 0.1551 - acc: 0.9467 - val_loss: 0.1473 - val_acc: 0.9465
Out[161]:

TFIDF + SGD

SGD: Accuracy mean 0.956443096548 +/- 0.00297615337351
CPU times: user 1.93 s, sys: 307 ms, total: 2.24 s
Wall time: 4min 31s


GNews word2vec 300d

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 398s - loss: 1.1628 - acc: 0.6303 - val_loss: 0.1695 - val_acc: 0.9480
Epoch 2/2
15998/15998 [==============================] - 414s - loss: 0.1640 - acc: 0.9469 - val_loss: 0.1440 - val_acc: 0.9470

```



In [3]:
from __future__ import print_function
import os
import numpy as np
np.random.seed(1337)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

Using Theano backend.


In [4]:
BASE_DIR = ''
GLOVE_DIR = BASE_DIR + '/Users/laam/pretrained/'
TEXT_DATA_DIR = BASE_DIR + './20_newsgroup/'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [5]:
print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

Indexing word vectors.


In [6]:
embeddings_index['hanoi'].shape

(100,)

In [7]:
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                f = open(fpath)
                texts.append(f.read())
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


In [9]:
print(texts[0])
print(labels[0])
print(labels_index)

Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew
From: mathew <mathew@mantis.co.uk>
Newsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers
Subject: Alt.Atheism FAQ: Atheist Resources
Summary: Books, addresses, music -- anything related to atheism
Keywords: FAQ, atheism, books, music, fiction, addresses, contacts
Message-ID: <19930329115719@mantis.co.uk>
Date: Mon, 29 Mar 1993 11:57:19 GMT
Expires: Thu, 29 Apr 1993 11:57:19 GMT
Followup-To: alt.atheism
Distribution: world
Organization: Mantis Consultants, Cambridge. UK.
Approved: news-answers-request@mit.edu
Supersedes: <19930301143317@mantis.co.uk>
Lines: 290

Archive-name: atheism/resources
Alt-atheism-archive-name: resources
Last-modified: 11 December

In [10]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 214909 unique tokens.


In [11]:
np.min(word_index.values())

1

In [12]:
word_index['bbc']

17507

In [13]:
sequences[0][:10]

[169, 29, 24, 16, 12, 2, 127, 357, 127, 357]

In [14]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [15]:
data[0][:10]

array([ 119,  489,  755,   76,  697, 2361,   10,   76,  118, 3006], dtype=int32)

In [16]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [17]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [18]:
nb_validation_samples

3999

In [19]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [20]:
x_train[0][:100], y_train[0]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.]))

In [21]:
print('Preparing embedding matrix.')
nb_words = min(MAX_NB_WORDS, len(word_index))
# prepare embedding matrix
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [22]:
embedding_matrix[18895]

array([-0.29572999, -0.36605   , -0.50216001, -0.10307   ,  0.28707999,
       -0.23991001, -0.50765997,  0.27495   ,  0.85957998, -0.54860997,
        0.27494001,  0.96930999, -0.15655001,  0.61331999,  0.16321   ,
       -0.75143999,  0.27669999, -0.49566999,  0.44659001,  0.40711999,
        0.85280001,  0.46781   ,  0.11448   ,  0.36669999, -0.15640999,
        0.62299001,  1.07509995, -0.20028999, -0.61505002,  0.10497   ,
       -0.58802998,  0.29655001,  0.42434001,  0.36337   , -0.24229001,
        0.24217001,  0.67123002,  1.07869995,  0.34046   , -0.67088997,
       -0.51059997, -0.11725   , -0.14871   , -0.40559   , -0.093417  ,
        0.16556001, -0.11101   , -0.66891998,  0.69362003, -0.14394   ,
        0.077842  , -0.62276   , -0.31762001,  0.34355   , -0.58569998,
       -0.011997  , -0.34718001,  0.72273999,  0.27320999,  0.38378   ,
        0.21027   , -0.06441   ,  0.19183999,  0.56654   , -0.47075999,
        0.27746001,  0.19335   ,  0.55070001, -0.57810003,  1.10

In [29]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(nb_words + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [26]:
# no pretrained embedding
embedding_layer = Embedding(nb_words + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)


In [38]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x) # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Training model.


In [39]:
# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=2, batch_size=64, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 217s - loss: 1.1108 - acc: 0.6467 - val_loss: 0.1907 - val_acc: 0.9395
Epoch 2/2
15998/15998 [==============================] - 207s - loss: 0.1458 - acc: 0.9489 - val_loss: 0.1498 - val_acc: 0.9450


In [ ]:
from keras.utils.visualize_util import plot
plot(model, to_file='model.png')

In [ ]:
from IPython.display import Image

In [ ]:
Image("./model.png")

## Oldies but Goodies

In [ ]:
# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                f = open(fpath)
                texts.append(f.read())
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

In [ ]:
texts[0], labels[0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import StratifiedKFold, cross_val_score

In [ ]:
vectorizer = TfidfVectorizer(
                    encoding='utf-8',
                    decode_error='replace', 
                    strip_accents='unicode', 
                    lowercase=True, 
                    analyzer='word', 
                    stop_words='english',
                    token_pattern=r'(?u)\b[a-zA-Z_][a-zA-Z0-9_]+\b', #no special tags
                    max_df=0.5, min_df=2,
                    ngram_range=(1, 2), 
                    max_features = MAX_NB_WORDS,
                    norm='l2', 
                    use_idf=True, 
                    smooth_idf=True, 
                    sublinear_tf=True)

In [ ]:
X = vectorizer.fit_transform(texts)

In [ ]:
y = labels

In [ ]:
X.shape, len(y)

In [ ]:
sgd = SGDClassifier(loss="modified_huber", penalty="elasticnet", n_iter=240, n_jobs=4, class_weight='balanced')

In [ ]:
%%time

sgd_scores = cross_val_score(sgd,  # steps to convert raw messages into models
                         X,  # training data
                         y,  # training labels
                         cv=StratifiedKFold(y, n_folds=10, shuffle=True), 
                         scoring='accuracy', 
                         n_jobs=4
                         )

print("SGD: F1 mean {} +/- {}".format(np.mean(sgd_scores), np.std(sgd_scores)))

## Using Google news 300d

In [40]:
from gensim.models import Word2Vec

In [41]:
w2v = Word2Vec.load_word2vec_format("/Users/laam/pretrained/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [42]:
EMBEDDING_DIM = 300

In [61]:
print('Preparing embedding matrix.')
nb_words = min(MAX_NB_WORDS, len(word_index))
# prepare embedding matrix
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        if i > MAX_NB_WORDS: continue
        embedding_vector = np.asarray(w2v[word], dtype='float32')
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    except Exception, e:
        print("ERROR", e)

Preparing embedding matrix.
ERROR 'rscharfy'
ERROR 'thrace'
ERROR '270'
ERROR '271'
ERROR '272'
ERROR '273'
ERROR '274'
ERROR '275'
ERROR '276'
ERROR '277'
ERROR '278'
ERROR '279'
ERROR '27k'
ERROR 'tredysvr'
ERROR 'dnd'
ERROR 'benedikt'
ERROR 'beyer'
ERROR 'fij'
ERROR 'xuserfilesearchpath'
ERROR '6ql'
ERROR 'ahmet'
ERROR 'irq'
ERROR 'utcsri'
ERROR 'c5jrde'
ERROR 'mcontent'
ERROR 'unbvm1'
ERROR 'zeos'
ERROR 'mentorg'
ERROR 'larrison'
ERROR 'mangoe'
ERROR 'cambodia'
ERROR 'omran'
ERROR 'olympus'
ERROR 'mach1'
ERROR 'bowmanj'
ERROR 'tweekco'
ERROR 'pettefar'
ERROR 'telix'
ERROR 'hamer'
ERROR 'vuw'
ERROR 'cunyvm'
ERROR 'crossposting'
ERROR 'motecc'
ERROR 'ghostview'
ERROR '199'
ERROR '198'
ERROR '195'
ERROR '194'
ERROR '197'
ERROR '196'
ERROR '191'
ERROR '190'
ERROR '193'
ERROR '192'
ERROR 'slmr'
ERROR 'uxa'
ERROR '617'
ERROR 'dangermouse'
ERROR 'vishnepolsky'
ERROR 'wrldlnk'
ERROR 'fv'
ERROR 'stuffit'
ERROR 'sccsi'
ERROR 'olerud'
ERROR 'diffie'
ERROR 'prescod'
ERROR 'geoffrey'
ERROR 'alp

In [65]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(nb_words + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [66]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Training model.


In [67]:
# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=2, batch_size=128, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 398s - loss: 1.1628 - acc: 0.6303 - val_loss: 0.1695 - val_acc: 0.9480
Epoch 2/2
15998/15998 [==============================] - 414s - loss: 0.1640 - acc: 0.9469 - val_loss: 0.1440 - val_acc: 0.9470
